In [13]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window
import talib

In [14]:
if sc.master[0:5]=="local":
    #Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180511"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180511"
    Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180518"
else:
    Path = "hdfs://master:9000/user/hduser"
Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180518"

In [15]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [16]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [17]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 19305
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50     

In [18]:
#取出etf的distinct id
etf_ids = []
for row in tetfp_dt.select("etf_id").distinct().orderBy("etf_id").collect():
    etf_ids.append(row["etf_id"])
etf_ids

['0050   ',
 '0051   ',
 '0052   ',
 '0053   ',
 '0054   ',
 '0055   ',
 '0056   ',
 '0057   ',
 '0058   ',
 '0059   ',
 '006201 ',
 '006203 ',
 '006204 ',
 '006208 ',
 '00690  ',
 '00692  ',
 '00701  ',
 '00713  ']

In [19]:
etf_dic = {}
for etfid in etf_ids:
    export_dt = tetfp_dt.filter("etf_id='" +etfid+ "'").orderBy("etf_id", "etf_date", ascending=True)
    export_pd = export_dt.toPandas()
    etf_dic.update({etfid.strip(): export_pd})

In [20]:
#functions of TA lib
print(talib.get_functions())

['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'ADD', 'DIV', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'MULT', 'SUB', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR',

In [21]:
etf_dic["0050"]["etf_close"].count()

1317

In [47]:
import math
def get_feature_pre(series):
    #在feature前加上一個Nan後回傳
    rtn_list = series.tolist()
    rtn_list = [None] + rtn_list
    return rtn_list[:-1]

def checkNan(num):
    if num == None:
        return True
    elif math.isnan(num):
        return True
    else:
        return False
    
def calculate_n_bias(close_price, ma):
#     (close price - MA)/MA
    if checkNan(close_price) or checkNan(ma):
        return None
    else:
        return (close_price - ma)/ma
    
def calculate_n_bias_list(close_ser, ma_list):
    #calculate n bias (20 days)
    bias_list = []
    for cprice, ma in zip(get_feature_pre(close_ser), ma_list):
        bias_list.append(calculate_n_bias(cprice, ma))
    return bias_list

def calculate_bias_ma_list(ma3_list, ma6_list):
    #calculate bias_ma = (ma3-ma6)/ma6
    bias_list = []
    for ma3, ma6 in zip(ma3_list, ma6_list):
        bias_list.append(calculate_n_bias(ma3, ma6))
    return bias_list

def calculate_bias_3_6_list(bias3_list, bias6_list):
    #bias_3_6 = bias3 - bias6
    bias_list = []
    for bias3, bias6 in zip(bias3_list, bias6_list):
        if checkNan(bias3) or checkNan(bias6):
            bias_list.append(None)
        else:
            bias_list.append(bias3 - bias6)
    return bias_list

def calculate_dif(ema12_list, ema26_list):
    #差離值DIF = 12日EMA - 26日EMA 
    dif_list = []
    for ema12, ema26 in zip(ema12_list, ema26_list):
        if checkNan(ema12) or checkNan(ema26):
            dif_list.append(None)
        else:
            dif_list.append(ema12 - ema26)
    return dif_list

def create_feature(etf_pd):
    close_ser = etf_pd["etf_close"]
    #EMA
    ema5=get_feature_pre(talib.EMA(close_ser,timeperiod=5))
    ema10=get_feature_pre(talib.EMA(close_ser,timeperiod=10))
    ema20=get_feature_pre(talib.EMA(close_ser,timeperiod=20))
    ema30=get_feature_pre(talib.EMA(close_ser,timeperiod=30))
    #BIAS
    #nBIAS -3, 6, 20 => (close price - MA)/MA   ,Paper 建議用20日MA
    ma3 = get_feature_pre(talib.MA(np.array(close_ser), timeperiod=3))
    ma6 = get_feature_pre(talib.MA(np.array(close_ser), timeperiod=6))
    ma20 = get_feature_pre(talib.MA(np.array(close_ser), timeperiod=20))
    nbias3 = calculate_n_bias_list(close_ser, ma3)
    nbias6 = calculate_n_bias_list(close_ser, ma6)
    nbias20 = calculate_n_bias_list(close_ser, ma20)
    
    #BIAS_ma = (ma3-ma6)/ma6
    bias_ma = calculate_bias_ma_list(ma3, ma6)
    
    #bias_3_6 = bias3 - bias6
    bias_3_6 = calculate_bias_3_6_list(nbias3, nbias6)
    
    #KD --> only use STOCHRSI
    k, d = talib.STOCHRSI(close_ser, timeperiod=9, fastk_period=3, fastd_period=3, fastd_matype=1)
    k = get_feature_pre(k)
    d = get_feature_pre(d)
    
    #差離值DIF = 12日EMA - 26日EMA 
    ema12=get_feature_pre(talib.EMA(close_ser,timeperiod=12))
    ema26=get_feature_pre(talib.EMA(close_ser,timeperiod=26))
    dif = calculate_dif(ema12, ema26)
    
    #MACD
    macd, macdsignal, macdhist = talib.MACD(close_ser, fastperiod=12, slowperiod=26, signalperiod=9)
    macd = get_feature_pre(macd)
    macdsignal = get_feature_pre(macdsignal)
    macdhist = get_feature_pre(macdhist)
    
    #RSI
    rsi = get_feature_pre(talib.RSI(close_ser, timeperiod=10))
    
    #CMO
    cmo = get_feature_pre(talib.CMO(close_ser, timeperiod=14))
    
    #ROCR - Rate of change ratio: (price/prevPrice)
    rocr = get_feature_pre(talib.ROCR(close_ser, timeperiod=10))
    
    #PPO - Percentage Price Oscillator
    ppo = get_feature_pre(talib.PPO(close_ser, fastperiod=12, slowperiod=26, matype=0))
    
    rtn_pd = pd.DataFrame({"etf_id": etf_pd["etf_id"], "etf_date": etf_pd["etf_date"], "etf_close": etf_pd["etf_close"], 
                           "ema5": ema5, "ema10": ema10, "ema20": ema20, "ema30" : ema30,
                          "nbias3": nbias3, "nbias6": nbias6, "nbias20": nbias20, "bias_ma": bias_ma,
                          "bias_3_6": bias_3_6, "k": k, "d": d, "dif": dif,
                          "macd": macd, "macdsignal": macdsignal, "macdhist": macdhist,
                          "rsi": rsi, "cmo": cmo, "rocr": rocr, "ppo": ppo})
 
    return rtn_pd

In [46]:
rsi = talib.RSI(etf_dic["0050"]["etf_close"], timeperiod=10)
rsi

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
5             NaN
6             NaN
7             NaN
8             NaN
9             NaN
10      37.931034
11      32.890365
12      45.876749
13      45.130324
14      47.953042
15      47.056459
16      41.841079
17      38.666624
18      44.513467
19      57.893389
20      59.345729
21      59.345729
22      62.536336
23      68.099830
24      62.909653
25      68.281634
26      70.643752
27      71.238724
28      76.085539
29      65.228944
          ...    
1287    38.460531
1288    45.596268
1289    48.443042
1290    51.718058
1291    48.308433
1292    48.308433
1293    46.161488
1294    37.768796
1295    40.442439
1296    52.377488
1297    37.334862
1298    32.912684
1299    30.880630
1300    29.611041
1301    28.317474
1302    30.346043
1303    40.349436
1304    36.036590
1305    30.846677
1306    35.689482
1307    41.905071
1308    47.072196
1309    51.395239
1310    53.279696
1311    61

In [48]:
#EMA
create_feature(etf_dic["0050"])

,bias_3_6,bias_ma,cmo,d,dif,ema10,ema20,ema30,ema5,etf_close,...,k,macd,macdhist,macdsignal,nbias20,nbias3,nbias6,ppo,rocr,rsi
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.25,...,NaN,NaN,NaN,NaN,NaN,-1.526718e-03,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.90,...,NaN,NaN,NaN,NaN,NaN,-5.195599e-03,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.380000,54.10,...,NaN,NaN,NaN,NaN,NaN,-5.840762e-03,NaN,NaN,NaN,NaN
6,0.004603,-0.004601,NaN,NaN,NaN,NaN,NaN,NaN,54.286667,54.50,...,NaN,NaN,NaN,NaN,NaN,3.081664e-04,-0.004294,NaN,NaN,NaN
7,0.003394,-0.003373,NaN,NaN,NaN,NaN,NaN,NaN,54.357778,54.45,...,NaN,NaN,NaN,NaN,NaN,6.153846e-03,0.002760,NaN,NaN,NaN
8,-0.001230,0.001228,NaN,NaN,NaN,NaN,NaN,NaN,54.388519,54.50,...,NaN,NaN,NaN,NaN,NaN,1.839926e-03,0.003070,NaN,NaN,NaN
9,-0.003685,0.003684,NaN,NaN,NaN,NaN,NaN,NaN,54.425679,54.00,...,NaN,NaN,NaN,NaN,NaN,3.059039e-04,0.003991,NaN,NaN,NaN
